In [1]:
from wikipedia import WikipediaPage
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
import networkx as nx
plt.style.use('ggplot') 
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import warnings
warnings.simplefilter('ignore')

In [19]:
def find_members(content):
    cmem = '== Current members =='
    bmem = '== Band members =='
    mem = '== Members =='
    if cmem in content:
        member_section = content.split(cmem)[1].split('=')[0]
    elif bmem in content:
        member_section = content.split(bmem)[1].split('=')[0]
    elif mem in content:
        member_section = content.split(mem)[1].split('=')[0]
    else:
        member_section = ''
    return member_section

def get_member_section(band):
    try:
        try:
            band_page = WikipediaPage(title=band+ ' (band)')
        except:
            try:
                band_page = WikipediaPage(title='The '+band+' (band)')
            except:
                try:
                    band_page = WikipediaPage(title='The '+band)
                except:
                    band_page = WikipediaPage(title=band)
        member_section = find_members(band_page.content)
        return member_section
    except:
        return ''

def get_member_list(band):
    member_section = get_member_section(band)
    members = [x.split('–')[0].strip() for x in member_section.split('\n') if '–' in x]
    members_clean = []
    final = 'Final line-up'
    for member in members:
        if final in member: member=member.replace(final,'')
        members_clean.append(member)
    return members_clean

def get_discog_bands(member, main_band):
    band_ident = re.compile(r'===.*===')
    try:
        page_contents = WikipediaPage(title=member).content
        discog_section = page_contents.split('== Discography ==')[1].split('\n== ')[0]
        bands_raw = re.findall(band_ident,discog_section)
        bands=[]
        for band in bands_raw:
            band = band.replace('=','').strip()
            if band.lower()[:4] == 'with':
                band = band[4:].strip()
            check_passed=True
            for check in ['/',
                          'solo',
                          'compilation',
                          'collaboration',
                          'miscellaneous',
                          'producing',
                          'producer',
                          'studio',
                          'guest',
                          'singles']:
                if check in band.lower(): check_passed=False
            for check in ['EPs']:
                if check in band: check_passed=False
            if check_passed and band.lower() != main_band.lower():
                bands.append(band)
        return bands
    except:
        return ''
    
def get_related_bands(band):
    related = {}
    members = get_member_list(band).copy()
    if members:
        for member in members:
            related[member]=get_discog_bands(member, band)
        return related
    else:
        return 'Band not found'

In [3]:
def build_graph(band):
    band_related = get_related_bands(band)
    band_nodes = [band] + [sub_band for band_list in band_related.values() for sub_band in band_list]
    G = nx.Graph()
    G.add_nodes_from(band_nodes)
    for member, bands in band_related.items():
        for sub_band in bands:
            G.add_edge(band,sub_band,name=member)
    return G

In [4]:
def draw_graph(G):
    labels = {a:b for a,b in zip(G.nodes,[x.replace(' ','\n') for x in G.nodes])}
    fig, ax = plt.subplots(figsize=(10,10))
    nx.draw(G,
                         labels=labels,
    #                      node_size=node_sizes,
                         node_size=4000,
                         node_shape='o',width=1,
                         font_color='white',font_weight='bold',
                         ax=ax)
    plt.show();

In [5]:
def plot_related(band):
    try:
        graph = build_graph(band)
        draw_graph(graph)
    except:
        print('Not found')

In [21]:
plot_related('Blur')

Not found


In [22]:
print(WikipediaPage(title='Blur (band)').content)

Blur are an English rock band, formed in London in 1988. The group consists of singer/keyboardist Damon Albarn, guitarist/singer Graham Coxon, bassist Alex James and drummer Dave Rowntree. Their debut album Leisure (1991) incorporated the sounds of Madchester and shoegazing. Following a stylistic change influenced by English guitar pop groups such as the Kinks, the Beatles and XTC, Blur released Modern Life Is Rubbish (1993), Parklife (1994) and The Great Escape (1995). In the process, the band became central to the Britpop music and culture movement, and achieved mass popularity in the UK, aided by a chart battle with rivals Oasis in 1995 dubbed the "Battle of Britpop".
In recording their follow-up, Blur (1997), the band underwent another reinvention, showing influence from the lo-fi style of American indie rock groups. The band's third UK number one album, Blur included the "Song 2" single, which brought them mainstream success in the US. Their next album, 13 (1999) saw the band memb